In [7]:
import torch
import time
torch.__version__

'2.0.1+cpu'

## 1. Create Tensor Directly on the Target Device

#### Don't

In [8]:
start_time = time.time()
for _ in range(100):
	# Create on the CPU, then transfering to the GPU
	cpu_tensor = torch.ones((1000, 64, 64))
	gpu_tensor = cpu_tensor.cuda()
end_time = time.time()
print('Total time: {:.3f}s'.format(end_time - start_time))

AssertionError: Torch not compiled with CUDA enabled

#### Do

In [ ]:
start_time = time.time()
for _ in range(100):
	# Create directly on the GPU
	gpu_tensor = torch.ones((1000, 64, 64), device='cuda')
end_time = time.time()
print('Total time: {:.3f}s'.format(end_time - start_time))

AssertionError: Torch not compiled with CUDA enabled

## 2. Use Sequential Layers When Possible

#### Don't

In [ ]:
class ExampleModelDont(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        input_size      = 2
        output_size     = 3
        hidden_size     = 16

        self.inputlayer         = torch.nn.Linear(input_size, hidden_size)
        self.inputactivation    = torch.nn.ReLU()

        self.midlayer           = torch.nn.Linear(hidden_size, hidden_size)
        self.midactivation      = torch.nn.ReLU()

        self.outputlayer        = torch.nn.Linear(hidden_size, output_size)
        self.outputactivation   = torch.nn.Softmax()

    def foward(self, input):
        input           = self.inputlayer(input)
        input           = self.inputactivation(input)

        input           = self.midlayer(input)
        input           = self.midactivation(input)

        input           = self.outputlayer(input)
        output          = self.outputactivation(input)

        return output

#### Do

In [ ]:
class ExampleSequentialModel(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        input_size      = 2
        output_size     = 3
        hidden_size     = 16

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, output_size),
            torch.nn.Softmax()
        )

    def foward(self, input):
        output = self.layers(input)

        return output

## 3. Don't Make Lists of Layers

#### Bad Approach

In [ ]:
class BadListExample(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        input_size      = 2
        output_size     = 3
        hidden_size     = 16

        self.inputlayer         = torch.nn.Linear(input_size, hidden_size)
        self.inputactivation    = torch.nn.ReLU()

        # Common error when using list layers
        self.midlayers          = []
        for _ in range(5):
            self.midlayers.append(torch.nn.Linear(hidden_size, hidden_size))
            self.midlayers.append(torch.nn.ReLU())
        
        self.outputlayer        = torch.nn.Linear(hidden_size, output_size)
        self.outputactivation   = torch.nn.Softmax()

    def forward(self, input):
        input = self.inputlayer(input)
        input = self.inputactivation(input)

        for layer in self.midlayers:
            input = layer(input)

        input = self.outputlayer(input)
        out   = self.outputactivation(input)

        return out 

NOTE: Vấn đề khi sử dụng list model là khi ta đưa model và data vào GPU, list sẽ không được registry trong GPU -> xung đột device giữa model và data.

#### Better Approach

In [ ]:
class BadListExample(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        input_size      = 2
        output_size     = 3
        hidden_size     = 16

        self.inputlayer         = torch.nn.Linear(input_size, hidden_size)
        self.inputactivation    = torch.nn.ReLU()

        # Common error when using list layers
        self.midlayers          = []
        for _ in range(5):
            self.midlayers.append(torch.nn.Linear(hidden_size, hidden_size))
            self.midlayers.append(torch.nn.ReLU())

        # Fix list layers
        self.midlayers          = torch.nn.Sequential(*self.midlayers)

        self.outputlayer        = torch.nn.Linear(hidden_size, output_size)
        self.outputactivation   = torch.nn.Softmax()

    def forward(self, input):
        input = self.inputlayer(input)
        input = self.inputactivation(input)
        input = self.midlayers(input)
        input = self.outputlayer(input)
        out   = self.outputactivation(input)

        return out 

## 4. Make Use of Distributions

## 5. Use detach() On Long-Term Metrics

#### Don't

In [9]:
losses = []
for batch in data_batches:
    output = example_model(batch)

    target = torch.rand((10,3))
    loss = metric(output, target)
    losses.append(loss)

NameError: name 'data_batches' is not defined

#### Do

In [ ]:
losses = []
for batch in data_batches:
    output = example_model(batch)

    target = torch.rand((10,3))
    loss = metric(output, target)
    # Fix don't to do
    losses.append(loss.detach()) # or can use loss.item()

## 6. Trick to Delete a Model from GPU

In [ ]:
import gc
model = Model().cuda()

del model
gc.collect()
# The model will normally stay on the 
# cache until something takes it's place
torch.cuda.empty_cache()

## 7. Call eval() Before Testing

In [ ]:
model = Model()
# Training
model.train()
# End training
# Evaluating
model.eval()
# End evaluating